https://www.everydayhealth.com/conditions/

In [1]:
import re
import os
import requests
import hashlib
import json
import time
import random
import string
import pandas as pd
from tqdm import tqdm
import codecs
from glob import glob
from bs4 import BeautifulSoup
from scrapy.selector import Selector
from urllib.parse import unquote
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
# 定义随机的useragent
from fake_useragent import UserAgent
ua = UserAgent()

# 设置代理ip
ipdata = pd.read_excel('D:/Intern_Project/代理ip的地址2000.xlsx')
ipdata.fillna("",inplace = True)

proxieslist = []

for i in ipdata.index.values:
    line = ipdata.loc[i,['HTTP']].to_dict()
    proxieslist.append(line)
    
# randomproxy = random.choice(proxieslist)

# 设置消息头
headers_pc = {
    'User-Agent': ua.random,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
}

params = {
    'ie': 'UTF-8',
    'wd': 'Python'
}


In [3]:
def changeparams():
    headers_pc = {
    'User-Agent': ua.random,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
    }
    randomproxy = random.choice(proxieslist)
    
    return headers_pc, randomproxy


In [4]:
headers_pc, randomproxy = changeparams()
res = requests.get('https://www.everydayhealth.com', headers=headers_pc, proxies= randomproxy)
# 将cookieJar数据转化为字典
cookie_dict = requests.utils.dict_from_cookiejar(res.cookies)

In [8]:
def html_download(headers_pc, randomproxy, item):
    # pbar.set_description(item['term'])
    try:
        url = item['url']
        newterm = item['term'].strip().replace('/', ' ')
        if os.path.exists(f"D:/Intern_Project/5_everydayhealth/drugs_html/{newterm}.html"):
#         if os.path.exists(f"D:/Intern_Project/5_everydayhealth/health_html/{newterm}.html"):
#             pbar.update(1)    
            return None
#         time.sleep(random.randint(0, 5))
        r = requests.get(url, headers=headers_pc, proxies=randomproxy, cookies=cookie_dict, timeout=10)
        r.encoding = 'utf-8'
        html = r.text   
        
        with open(f"D:/Intern_Project/5_everydayhealth/drugs_html/{newterm}.html", 'w+', encoding='utf-8') as f:
            f.write(html)
            
#         pbar.update(1)
    
    except Exception as e:
        print(e)
        headers_pc, randomproxy = changeparams()
        time.sleep(random.randint(5,10))
        html_download(headers_pc, randomproxy, item)
    return None

In [13]:
def task_monitor(task_list, mt_pool):
    while len(task_list) > 0:
        tmp_task_list = []
        for future in task_list:
            if future.done():
                result = future.result()
#                 results.extend(result)
            else:
                tmp_task_list.append(future)
        
        task_list = tmp_task_list

In [ ]:
# 下载drugs_html
with open('D:/Intern_Project/5_everydayhealth/drugs术语链接列表.json', 'r', encoding='utf-8') as f:
    url_list = json.load(f)

pbar = tqdm(total=len(url_list))
all_craw_tasks = []
headers_pc, randomproxy = changeparams()
with ThreadPoolExecutor(max_workers=10) as mt_pool:
    for item in url_list:
        headers_pc, randomproxy = changeparams()
        tmp_task = mt_pool.submit(html_download, headers_pc, randomproxy, item)
        all_craw_tasks.append(tmp_task)

    # 监测下载任务进展
    task_monitor(all_craw_tasks, mt_pool)

  8%|██████▌                                                                          | 1129/13815 [00:10<3:08:30,  1.12it/s]

HTTPSConnectionPool(host='www.everydayhealth.com', port=443): Read timed out. (read timeout=10)


 30%|████████████████████████▉                                                          | 4141/13815 [07:52<33:20,  4.84it/s]

HTTPSConnectionPool(host='www.everydayhealth.com', port=443): Read timed out. (read timeout=10)


 30%|████████████████████████▉                                                          | 4148/13815 [07:54<28:16,  5.70it/s]

...

 44%|████████████████████████████████████▍                                              | 6072/13815 [12:27<32:02,  4.03it/s]

HTTPSConnectionPool(host='www.everydayhealth.com', port=443): Read timed out. (read timeout=10)


 44%|████████████████████████████████████▌                                              | 6080/13815 [12:29<29:17,  4.40it/s]

HTTPSConnectionPool(host='www.everydayhealth.com', port=443): Read timed out.


 65%|█████████████████████████████████████████████████████▌                             | 8913/13815 [18:27<36:21,  2.25it/s]

In [9]:
with open('D:/Intern_Project/5_everydayhealth/drugs术语链接列表.json', 'r', encoding='utf-8') as f:
    url_list = json.load(f)
headers_pc, randomproxy = changeparams()
count = 0
for item in url_list:  
    headers_pc, randomproxy = changeparams()
    html_download(headers_pc, randomproxy, item)
    count += 1
    if count % 100 == 0:
        print(count)
        time.sleep(random.randint(5,10))
        headers_pc, randomproxy = changeparams()
        res = requests.get('https://www.everydayhealth.com', headers=headers_pc, proxies= randomproxy)
        # 将cookieJar数据转化为字典
        cookie_dict = requests.utils.dict_from_cookiejar(res.cookies)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
HTTPSConnectionPool(host='www.everydayhealth.com', port=443): Read timed out. (read timeout=10)
HTTPSConnectionPool(host='www.everydayhealth.com', port=443): Read timed out. (read timeout=10)
HTTPSConnectionPool(host='www.everydayhealth.com', port=443): Read timed out. (read timeout=10)
HTTPSConnectionPool(host='www.everydayhealth.com', port=443): Read timed out. (read timeout=10)
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
1210

In [19]:
def html_download_append(headers_pc, randomproxy, url):
   # pbar.set_description(item['term'])
    try:
#         if os.path.exists(f"D:/Intern_Project/5_everydayhealth/drugs_html/{newterm}.html"):
# #         if os.path.exists(f"D:/Intern_Project/5_everydayhealth/health_html/{newterm}.html"):
#             pbar.update(1)    
#             return None
#         time.sleep(random.randint(0, 5))
        item = url[31:].strip('/').replace('/', '--')
        r = requests.get(url, headers=headers_pc, proxies=randomproxy, cookies=cookie_dict, timeout=10)
        r.encoding = 'utf-8'
        html = r.text   
        
        with open(f"D:/Intern_Project/5_everydayhealth/health_html/{item}.html", 'w+', encoding='utf-8') as f:
            f.write(html)
            
        pbar.update(1)
    
    except Exception as e:
        print(e)
        headers_pc, randomproxy = changeparams()
        time.sleep(random.randint(5,10))
        html_download(headers_pc, randomproxy, url)
    return None

In [ ]:
# 下载health_html_append
with open('D:/Intern_Project/5_everydayhealth/补充文章列表.json', 'r', encoding='utf-8') as f:
    url_list_append = json.load(f)

# pbar = tqdm(total=len(url_list_append))

all_craw_tasks = []
headers_pc, randomproxy = changeparams()
with ThreadPoolExecutor(max_workers=10) as mt_pool:
    for url in url_list_append:
        headers_pc, randomproxy = changeparams()
        tmp_task = mt_pool.submit(html_download_append, headers_pc, randomproxy, url)
        all_craw_tasks.append(tmp_task)

    # 监测下载任务进展
    task_monitor(all_craw_tasks, mt_pool)

  1%|▋                                                                                     | 14/1870 [01:52<15:14,  2.03it/s]

HTTPSConnectionPool(host='www.everydayhealth.com', port=443): Max retries exceeded with url: /crohns-disease/self-injections-crohns/ (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None)))


  2%|█▌                                                                                    | 35/1870 [01:58<08:05,  3.78it/s]

string indices must be integers


  3%|██▍                                                                                   | 53/1870 [02:03<05:23,  5.62it/s]

...

string indices must be integers
string indices must be integers


 75%|██████████████████████████████████████████████████████████████▊                     | 1397/1870 [20:34<20:41,  2.63s/it]

string indices must be integers
string indices must be integers
string indices must be integersstring indices must be integers

string indices must be integers
string indices must be integers
string indices must be integers
HTTPSConnectionPool(host='www.everydayhealth.com', port=443): Max retries exceeded with url: /hs/living-with-prostate-cancer/spread-to-bones/ (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None)))
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
st

string indices must be integers
string indices must be integers
string indices must be integersstring indices must be integers

string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string i

string indices must be integers
string indices must be integers
string indices must be integersstring indices must be integers

string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string i

In [8]:
# 下载health_html
with open('D:/Intern_Project/5_everydayhealth/health术语链接列表.json', 'r', encoding='utf-8') as f:
    url_list = json.load(f)

pbar = tqdm(total=len(url_list))

all_craw_tasks = []
results = []
headers_pc, randomproxy, cookie_dict = changeparams()
with ThreadPoolExecutor(max_workers=10) as mt_pool:
    for item in url_list:
        tmp_task = mt_pool.submit(html_download, headers_pc, randomproxy, cookie_dict, item)
        all_craw_tasks.append(tmp_task)

    # 监测下载任务进展
    task_monitor(all_craw_tasks, mt_pool, results)

Arthroscopy:   6%|████▌                                                                   | 39/618 [00:19<02:04,  4.66it/s]

HTTPSConnectionPool(host='www.everydayhealth.com', port=443): Read timed out. (read timeout=10)
write() argument must be str, not None


Carbohydrate:  16%|███████████▎                                                           | 98/618 [00:38<03:31,  2.46it/s]

HTTPSConnectionPool(host='www.everydayhealth.comhttps', port=443): Max retries exceeded with url: //www.everydayhealth.com/pregnancy/all-articles/?ie=UTF-8&wd=Python (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000278D42BDEE0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
write() argument must be str, not None


Endocarditis:  30%|████████████████████▉                                                 | 185/618 [01:05<03:01,  2.38it/s]

...

HTTPSConnectionPool(host='www.everydayhealth.comhttps', port=443): Max retries exceeded with url: //www.everydayhealth.com/infectious-diseases/coronavirus/?ie=UTF-8&wd=Python (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000278D4219FD0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
write() argument must be str, not None


Sulfonylureas:  84%|██████████████████████████████████████████████████████████▎          | 522/618 [02:48<00:35,  2.70it/s]

[Errno 2] No such file or directory: 'D:/Intern_Project/5_everydayhealth/health_html/Staph/MRSA.html'


Zika:  98%|████████████████████████████████████████████████████████████████████████████▋ | 608/618 [03:13<00:03,  3.12it/s]

TypeError: 'NoneType' object is not iterable